In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from random import randint
import random
import numpy as np
import pandas as pd 
import torch.optim as optim
import os
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
import torch
from scipy.stats import vonmises
import numpy.fft as fft 
import math
res = 12

In [23]:
##Define the network to train on the data 
class CircleNet(nn.Sequential): 
    def __init__(self, res, dropout, n_out): 
        super(CircleNet, self).__init__()
        self.n_in = res
        self.affine1 = nn.Linear(self.n_in, 6)
        self.drop1 = nn.Dropout(dropout)
        ##self.affine2 = nn.Linear(8, 4)
        ##self.drop2 = nn.Dropout(dropout)
        self.affine3 = nn.Linear(6, n_out)
    def forward(self, x): 
        x = F.relu(self.affine1(x))
        x = self.drop1(x)
        ##x = F.relu(self.affine2(x))
        ##x = self.drop2(x)
        x = F.relu(self.affine3(x))
        x = F.softmax(x, dim = 1)
        return x

This is the Data Generation Code


In [5]:
## make some uniform circular data
std = 2.76
mean = 8.5
res = 12
num_points = 100
num_samples = 100
data = np.zeros([num_points,res + 1])
for i in range(num_points): 
    for j in range(num_samples): 
        num = randint(0,res-1)
        data[i][num]+= 1
        data[i][res] = -1
    data[i][:-1] = (data[i][:-1]-mean)/std
np.savetxt("uniform12ptdata100test.csv", data, delimiter = ",")

In [19]:
##Make Data for variable number of samples
num_points = 2500
res = 12
data = np.zeros([num_points, res+1])

std_factor = math.sqrt(1/res*(1-1/res))
for i in range(num_points):
    num_samples = randint(10,110)
    mean = num_samples/res
    for j in range(num_samples): 
        num = randint(0,res - 1)
        data[i][num] += 1
    data[i][res] = -1
    data[i][:-1] = (data[i][:-1] - mean)/std_factor/math.sqrt(num_samples)
np.savetxt("uniformvariablenumtest.csv", data, delimiter = ",")

In [71]:
##Make Von Mises for variable number of samples
num_points = 2500
data = np.zeros([num_points,res+1])
std_factor = math.sqrt(1/res*(1-1/res))

for i in range(num_points):
    SAMPLE_SIZE = randint(10,110)
    kappa = 1
    rotate_amt = random.random()*2*np.pi
    rotate_amt2 = random.random()*2*np.pi
    mean = SAMPLE_SIZE/res
    vm_dist = vonmises.rvs(kappa, loc = rotate_amt, size=SAMPLE_SIZE)
  
    for l in range(SAMPLE_SIZE):
        vm_dist[l] = ((vm_dist[l] + np.pi)/(2*np.pi))%1*(res*10)//10
        
        data[i][int(vm_dist[l])] += 1
       
    ##rotate_amt = randint(0,res - 1)
    ##data[i,0:-1] = rotate(data[i,0:-1], rotate_amt)
    data[i][:-1] = (data[i][:-1] - mean)/std_factor/math.sqrt(SAMPLE_SIZE)
    data[i,-1] = -7

np.savetxt("vonmises12variabletest.csv", data, delimiter = ",")    

In [13]:
np.std(data[10][:-1])

1.213753094338272

In [113]:
data = np.zeros([10000,res+1])
for i in range(10000):
    SAMPLE_SIZE = 100
    kappa = 1
    rotate_amt = random.random()*2*np.pi
    vm_dist = vonmises.rvs(kappa, loc = rotate_amt, size=SAMPLE_SIZE)
    for l in range(SAMPLE_SIZE):
        random_rotate_factor = randint(0,1)
        vm_dist[l] = ((vm_dist[l] + np.pi + np.pi*random_rotate_factor)/(2*np.pi))%1*(res*10)//10
    
    for k in range(SAMPLE_SIZE):
        data[i][int(vm_dist[k])] += 1
    ##rotate_amt = randint(0,res - 1)
    ##data[i,0:-1] = rotate(data[i,0:-1], rotate_amt)
    data[i][:-1] = (data[i][:-1] - 12)/5
    data[i,-1] = -6
np.savetxt("bimodal8pt100.csv", data, delimiter = ",")    

In [56]:
data = np.zeros([10000,res+1])
std = 3.7
mean = 8.5
for i in range(10000):
    SAMPLE_SIZE = 100
    kappa = 1.5
    rotate_amt = random.random()*2*np.pi
    rotate_amt2 = random.random()*2*np.pi
    vm_dist = vonmises.rvs(kappa, loc = rotate_amt, size=SAMPLE_SIZE//2)
    vm_dist2 = vonmises.rvs(kappa, loc = rotate_amt2, size = SAMPLE_SIZE//2)
    for l in range(SAMPLE_SIZE//2):
        vm_dist[l] = ((vm_dist[l] + np.pi)/(2*np.pi))%1*(res*10)//10
        vm_dist2[l] = ((vm_dist2[l] + np.pi)/(2*np.pi))%1*(res*10)//10
        data[i][int(vm_dist[l])] += 1
        data[i][int(vm_dist2[l])]+=1
    ##rotate_amt = randint(0,res - 1)
    ##data[i,0:-1] = rotate(data[i,0:-1], rotate_amt)
    data[i][:-1] = (data[i][:-1] - mean)/std
    data[i,-1] = -7
np.savetxt("vonmises12pt100.csv", data, delimiter = ",")    

In [83]:
data = np.zeros([100,res+1])
for i in range(100):
    SAMPLE_SIZE = 100
    kappa = 1.5
    rotate_amt = random.random()*2*np.pi
    vm_dist = vonmises.rvs(kappa, loc = rotate_amt, size=SAMPLE_SIZE)
   
      
    vm_dist = ((vm_dist + np.pi)/(2*np.pi))%1*(res*10)//10
    
    for k in range(SAMPLE_SIZE):
        data[i][int(vm_dist[k])] += 1
    ##rotate_amt = randint(0,res - 1)
    ##data[i,0:-1] = rotate(data[i,0:-1], rotate_amt)
    data[i][:-1] = (data[i][:-1] - 12)/5
    data[i,-1] = -3
np.savetxt("vonmises8ptnormalizedtest.csv", data, delimiter = ",")    

In [105]:
print(data[7])

[15. 14.  4. 18. 15. 11.  9. 14. -6.]


In [22]:
##This is a class that allows us to load data from csv files and make them into datasets that the dataloader can use
class CircularDataset(Dataset): 
    def __init__(self, data_root): 
        self.data = []
        self.dist_coder = LabelEncoder()
        self.dist_list = []
        for path in data_root: 
            array = pd.read_csv(path)
            
            self.dist_list += [array.values[0,-1]] ##get the distribution type for each file and add it to the list
            if len(self.data) == 0: 
                self.data = array.values
            else:
                print(self.data.shape)
                print(array.values.shape)
                self.data = np.concatenate((self.data,array.values), axis = 0)
        self.dist_coder.fit(self.dist_list)
    def __getitem__(self,idx): 
        return self.data[idx][0:-1], self.to_one_hot(self.dist_coder,([self.data[idx,-1]]))[0]
    def __len__(self): 
        return len(self.data)
    def to_one_hot(self, codec, values):
        value_idxs = codec.transform(values)
        return torch.eye(len(codec.classes_))[value_idxs]

This is the code where we train the network on the data

In [102]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.05)
rate = .005
batchsize = 100
dataset = CircularDataset(["uniformvariablenum.csv","vonmises12variable.csv"])
testdata = CircularDataset(["uniformvariablenumtest.csv", "vonmises12variabletest.csv"])

iterator = torch.utils.data.DataLoader(dataset, batch_size = batchsize, shuffle = True)
testiterator = torch.utils.data.DataLoader(testdata, batch_size = 1, shuffle = True)
net = CircleNet(12,0,2)
optimizer = optim.Adam(net.parameters(),lr = rate)
running_loss = 0
netlist = []
epochs = 12
net.apply(init_weights)
net.train()
loss_func = nn.CrossEntropyLoss()
for e in range(epochs): 
    print(running_loss)
    running_loss = 0
    accuracy = 0
    i = 0
    for dist, labels in iterator: 
        
        
        optimizer.zero_grad()
        out = net(dist.float())
        
        loss = loss_func(out,torch.max(labels, 1)[1])
       
           
        loss.backward()
        optimizer.step()
        running_loss += loss
    for dist, labels in testiterator: 
        ps = net(dist.float())

        i += 1
        top_p, top_class = ps.topk(1, dim=1)
       
        top_p2, top_class2 = labels.topk(1, dim = 1)
        
        equals = top_class == top_class2
        
        accuracy += torch.sum(equals.float())
        
    
    print(accuracy)
    print(i)
    accuracy = 0
    

(24999, 13)
(24999, 13)
(2499, 13)
(2499, 13)
0
tensor(4669.)
4998
tensor(226.5150, grad_fn=<AddBackward0>)
tensor(4671.)
4998
tensor(188.0112, grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [97]:
dataset = CircularDataset(["uniformvariablenum.csv", "vonmises12variable.csv"])
def sum(x):
    tot = 0
    for i in x: 
        tot += i
    return tot

for i in range(100):
    print(sum(np.abs(dataset.__getitem__(i+25000)[0])))

(24999, 13)
(24999, 13)
11.442241965007154
16.292006232647942
12.24850905318578
22.174955007381094
11.42180722349943
14.214106244380286
17.17503473597122
22.895017322155887
14.47254453973265
22.978250586152114
13.872143440592142
27.53803613810241
26.959391265929334
8.40778859694445
22.713854624858193
9.169118129500095
18.6864857780466
17.999468360038286
16.884635296354762
24.002673647866594
22.061669989311387
19.408970451090184
17.73829960078679
13.651115015868509
16.7695375995985
16.293950502149787
21.35756199910127
11.981699734455281
20.21340689200656
17.455720963551368
16.368338453422282
17.48765798551029
18.38860113768933
16.018171499325952
21.13668276453924
14.773819217309189
22.107176950648235
14.278612953602595
12.599357192051212
23.324546078343424
21.198348493652777
17.755920907481187
9.648363026488434
18.025518173236726
14.602277298806792
12.76358456347945
13.467214027353117
19.52280092011957
10.676577392256107
19.40874289332776
20.52450077260856
18.58490251248581
18.599485324

In [146]:
testdata = CircularDataset(["uniform12ptdata100.csv"])
testiterator = torch.utils.data.DataLoader(testdata, batch_size = 1, shuffle = True)
accuracy = 0

i = 0
for dist, labels in testiterator: 
    ps = net(dist.float())
        
    print(dist)
    i+=1
    top_p, top_class = ps.topk(1, dim=1)
    
    top_p2, top_class2 = labels.topk(1, dim = 1)
        
    equals = top_class == top_class2
   
    accuracy += torch.sum(equals.float())
        
print(accuracy/99)
print(i)


tensor([[-1.2162, -0.1351, -0.6757, -0.9459, -0.1351,  0.6757,  1.2162,  0.1351,
          0.1351, -0.1351,  0.4054,  0.1351]], dtype=torch.float64)
tensor([[ 0.1351, -0.6757,  0.6757, -0.4054, -1.4865, -0.4054, -0.9459, -1.2162,
          2.2973,  0.1351,  1.2162,  0.1351]], dtype=torch.float64)
tensor([[-0.4054, -0.1351,  1.2162,  0.4054, -0.4054, -0.4054, -0.1351,  0.4054,
         -0.1351,  0.1351, -0.9459, -0.1351]], dtype=torch.float64)
tensor([[-1.2162, -0.1351, -0.1351,  0.6757,  0.9459,  0.6757,  0.4054, -0.4054,
          0.6757, -0.4054, -0.9459, -0.6757]], dtype=torch.float64)
tensor([[ 0.1351,  0.6757, -0.4054,  0.1351, -0.9459,  0.1351,  1.2162, -0.9459,
          0.6757, -0.4054,  0.1351, -0.9459]], dtype=torch.float64)
tensor([[-0.6757, -1.2162,  0.1351, -0.6757,  2.2973,  0.1351,  0.9459, -0.1351,
          0.1351, -0.1351, -0.4054, -0.9459]], dtype=torch.float64)
tensor([[ 1.2162,  0.6757,  0.4054, -0.6757, -0.1351,  0.4054, -0.1351, -1.2162,
         -0.9459, -0.1351

         -1.7568,  0.4054, -0.1351,  0.6757]], dtype=torch.float64)
tensor([[-0.1351,  0.1351,  0.6757, -0.9459, -0.9459,  0.1351, -0.4054,  2.0270,
         -0.6757, -0.1351,  0.1351, -0.4054]], dtype=torch.float64)
tensor([[-0.4054, -0.9459,  0.9459, -1.7568,  0.4054,  0.1351, -0.1351,  0.9459,
          0.1351, -0.9459,  0.4054,  0.6757]], dtype=torch.float64)
tensor([[ 0.4054, -0.1351,  0.1351, -1.2162,  0.4054, -0.9459, -0.6757,  0.6757,
          0.9459,  0.1351, -0.6757,  0.4054]], dtype=torch.float64)
tensor([[ 0.1351,  0.6757,  1.4865,  0.9459, -0.9459,  0.1351,  0.1351, -0.6757,
          0.9459, -1.2162, -1.4865, -0.6757]], dtype=torch.float64)
tensor([[ 0.6757,  0.6757, -1.2162, -0.6757, -0.4054,  0.4054,  0.1351,  0.9459,
          0.9459, -0.6757, -1.2162, -0.1351]], dtype=torch.float64)
tensor([[ 0.1351, -0.4054, -0.9459, -0.4054, -0.6757, -0.4054,  0.1351, -0.6757,
         -0.9459,  2.2973,  1.2162,  0.1351]], dtype=torch.float64)
tensor([[ 0.9459, -0.1351, -0.9459,  0

          0.1351,  0.1351, -0.1351,  2.2973]], dtype=torch.float64)
tensor([[ 0.9459,  0.1351, -0.6757, -0.9459, -0.1351, -0.6757, -0.1351,  0.9459,
          0.1351,  0.1351, -0.1351, -0.1351]], dtype=torch.float64)
tensor([[-0.4054,  0.1351,  1.7568, -0.6757,  0.1351, -0.1351, -0.4054,  0.1351,
          0.4054, -0.6757, -1.2162,  0.4054]], dtype=torch.float64)
tensor([[ 0.1351,  0.9459, -0.6757, -1.4865,  0.4054, -0.4054, -0.4054,  0.1351,
         -0.1351, -0.6757,  0.9459,  0.6757]], dtype=torch.float64)
tensor([[ 0.1351, -0.4054,  0.1351, -0.4054,  2.0270,  0.4054, -1.2162,  0.1351,
         -0.1351, -1.4865,  0.1351,  0.1351]], dtype=torch.float64)
tensor([[-0.4054,  1.2162,  0.6757, -0.9459, -0.4054,  1.4865, -0.9459, -0.6757,
         -0.9459,  0.4054,  0.4054, -0.4054]], dtype=torch.float64)
tensor([[ 1.2162,  0.4054, -0.4054,  0.6757, -1.4865, -0.9459,  1.2162, -0.4054,
          0.1351, -0.9459,  0.6757, -0.6757]], dtype=torch.float64)
tensor([[ 0.9459,  0.9459, -0.1351,  0

          0.1351,  0.4054, -1.4865,  0.4054]], dtype=torch.float64)
tensor([[-0.1351,  0.4054, -0.1351, -1.2162, -0.1351, -0.6757, -1.2162,  0.4054,
          1.7568, -0.9459,  1.4865, -0.1351]], dtype=torch.float64)
tensor([[-0.4054, -1.4865, -0.4054, -0.1351,  1.7568, -0.1351,  0.1351, -0.6757,
          1.2162, -0.6757, -0.1351,  0.4054]], dtype=torch.float64)
tensor([[-0.6757, -0.6757,  0.9459, -0.6757, -0.1351,  0.6757,  0.4054,  0.1351,
         -0.4054, -1.2162,  0.4054,  0.6757]], dtype=torch.float64)
tensor([[ 0.4054, -0.9459,  1.7568, -0.6757, -0.1351, -0.1351, -0.6757,  0.1351,
         -0.1351,  0.1351, -0.9459,  0.6757]], dtype=torch.float64)
tensor([[ 0.1351,  0.4054,  0.4054, -1.7568,  0.6757,  1.7568, -0.9459, -1.7568,
          0.6757, -0.1351,  0.4054, -0.4054]], dtype=torch.float64)
tensor([[ 0.6757,  0.1351,  0.6757, -0.9459,  0.6757, -0.4054, -0.1351, -0.4054,
         -0.6757, -0.1351, -0.4054,  0.4054]], dtype=torch.float64)
tensor([[ 0.1351, -0.9459,  0.1351, -0

          0.9459, -0.1351, -0.1351, -0.6757]], dtype=torch.float64)
tensor([[ 0.1351,  0.4054, -0.4054, -0.1351,  0.4054, -0.6757,  0.1351,  0.1351,
          0.1351, -0.6757, -0.4054,  0.4054]], dtype=torch.float64)
tensor([[ 0.6757,  0.6757,  2.5676, -0.4054, -1.4865, -0.9459,  0.9459, -0.9459,
         -0.9459,  0.1351,  0.1351, -0.9459]], dtype=torch.float64)
tensor([[ 0.1351,  1.2162, -0.6757,  0.4054,  0.6757,  0.4054, -0.4054, -0.4054,
         -0.6757,  0.4054, -0.4054, -1.2162]], dtype=torch.float64)
tensor([[-0.1351,  0.1351,  0.4054,  0.1351,  1.2162,  0.9459, -0.1351, -0.9459,
         -0.4054, -1.7568, -0.1351,  0.1351]], dtype=torch.float64)
tensor([[-0.6757, -0.4054,  0.1351,  0.4054,  0.9459, -1.2162, -0.4054, -0.6757,
          1.2162,  0.4054, -0.6757,  0.4054]], dtype=torch.float64)
tensor([[ 0.9459,  1.2162,  0.1351, -1.2162,  0.4054,  1.4865, -0.4054, -1.2162,
         -1.2162,  1.2162, -0.9459, -0.9459]], dtype=torch.float64)
tensor([[-0.4054, -0.1351,  0.6757,  0

...

         -0.1351,  0.9459, -0.6757,  0.1351]], dtype=torch.float64)
tensor([[ 0.1351,  0.4054, -0.6757, -0.4054, -0.9459,  0.4054,  0.6757, -0.1351,
          1.4865,  0.1351, -1.4865, -0.1351]], dtype=torch.float64)
tensor([[ 0.1351, -0.6757, -0.4054,  0.4054, -0.9459,  1.7568, -0.1351, -0.4054,
         -0.9459,  0.9459,  0.1351, -0.4054]], dtype=torch.float64)
tensor([[-0.4054, -1.2162, -0.6757, -0.1351, -0.1351, -0.6757,  0.9459,  0.4054,
         -0.1351, -0.4054,  1.2162,  0.6757]], dtype=torch.float64)
tensor([[-0.6757,  0.4054,  1.2162, -0.4054,  0.9459, -0.4054,  0.1351,  0.1351,
         -0.6757, -1.2162, -0.6757,  0.6757]], dtype=torch.float64)
tensor([[-0.1351, -0.9459,  0.4054,  0.4054, -1.2162,  1.2162, -0.6757,  0.1351,
          0.4054,  0.6757, -0.1351, -0.6757]], dtype=torch.float64)
tensor([[-0.9459, -0.4054, -0.9459, -0.1351,  0.1351, -0.1351, -1.2162,  0.4054,
          2.5676, -0.6757,  1.2162, -0.4054]], dtype=torch.float64)
tensor([[ 0.4054, -0.6757,  0.4054,  0

          2.0270, -1.2162,  0.9459,  0.6757]], dtype=torch.float64)
tensor([[ 0.1351, -0.1351, -0.4054,  0.4054, -0.6757,  1.2162, -1.2162,  0.1351,
          0.9459,  0.6757, -0.6757, -0.9459]], dtype=torch.float64)
tensor([[-0.1351, -0.6757,  0.4054,  0.4054, -0.1351, -0.1351, -0.9459,  0.6757,
          0.4054, -0.4054, -0.1351,  0.1351]], dtype=torch.float64)
tensor([[-0.4054,  0.4054, -0.6757, -0.1351,  0.1351,  0.1351,  2.2973, -0.9459,
          0.4054, -1.4865,  1.2162, -1.4865]], dtype=torch.float64)
tensor([[ 0.4054, -0.4054,  1.4865,  0.6757, -0.4054,  0.1351, -0.6757, -1.4865,
          0.9459, -0.4054, -0.4054, -0.4054]], dtype=torch.float64)
tensor([[-0.1351, -0.6757,  0.4054, -0.4054,  0.6757, -0.4054,  0.6757, -0.6757,
         -0.6757,  0.1351,  0.9459, -0.4054]], dtype=torch.float64)
tensor([[-0.9459,  0.4054,  0.4054, -0.1351, -0.4054,  0.1351, -0.1351,  0.4054,
         -0.6757,  0.1351,  0.6757, -0.4054]], dtype=torch.float64)
tensor([[-0.4054,  0.1351,  0.1351,  0

         -1.4865,  0.1351,  0.9459,  0.4054]], dtype=torch.float64)
tensor([[ 0.1351,  0.1351, -0.6757,  1.7568, -1.4865, -0.6757,  0.4054, -0.1351,
          0.6757,  0.4054, -0.4054, -0.6757]], dtype=torch.float64)
tensor([[-0.9459,  0.4054, -0.4054, -1.2162,  0.1351, -1.2162,  1.2162, -0.4054,
          0.4054,  0.9459,  0.1351,  0.4054]], dtype=torch.float64)
tensor([[ 1.4865, -0.4054, -0.6757,  0.4054, -0.1351,  0.1351, -0.4054,  0.4054,
          0.1351, -0.9459, -0.1351, -0.4054]], dtype=torch.float64)
tensor([[ 0.1351, -0.4054,  0.6757,  0.6757, -0.1351,  0.1351, -0.4054, -0.6757,
          0.1351,  0.4054, -0.4054, -0.6757]], dtype=torch.float64)
tensor([[-0.9459,  0.1351,  0.1351, -1.2162,  0.9459,  0.6757, -0.1351, -0.1351,
         -0.9459, -0.6757, -0.1351,  1.7568]], dtype=torch.float64)
tensor([[ 0.1351,  0.9459, -0.4054,  0.1351, -0.4054,  0.4054, -0.6757,  0.1351,
         -0.4054,  0.4054,  0.4054, -1.2162]], dtype=torch.float64)
tensor([[-0.9459,  0.1351, -0.6757, -1

         -0.1351,  0.4054,  0.1351,  0.1351]], dtype=torch.float64)
tensor([[ 0.4054, -1.4865, -0.4054,  1.7568, -0.4054, -0.4054, -0.9459,  0.9459,
         -0.4054, -0.6757,  0.9459,  0.1351]], dtype=torch.float64)
tensor([[-0.4054, -0.1351, -0.9459, -0.4054,  0.9459,  0.9459,  0.1351,  0.4054,
         -0.6757, -0.1351,  0.1351, -0.4054]], dtype=torch.float64)
tensor([[ 0.1351,  0.1351,  1.7568,  0.6757, -0.4054, -0.6757, -0.1351, -1.4865,
         -0.6757, -0.1351,  0.6757, -0.4054]], dtype=torch.float64)
tensor([[-1.7568, -0.4054, -0.1351, -0.1351,  0.6757,  0.6757,  0.1351,  0.1351,
          0.4054, -0.6757, -0.1351,  0.6757]], dtype=torch.float64)
tensor([[-0.6757,  0.1351,  0.1351,  0.1351,  0.1351, -0.9459, -0.9459,  0.4054,
         -0.1351,  0.1351, -0.4054,  1.4865]], dtype=torch.float64)
tensor([[-0.4054,  0.6757,  0.1351,  0.6757,  0.4054, -0.4054, -0.4054, -0.4054,
         -0.4054, -0.1351, -0.9459,  0.6757]], dtype=torch.float64)
tensor([[-0.4054, -0.6757,  0.6757, -0

         -0.1351,  0.6757, -1.4865,  0.4054]], dtype=torch.float64)
tensor([[-0.4054, -0.9459, -0.9459, -0.9459,  0.9459, -0.1351,  0.4054,  0.4054,
          1.4865, -0.4054, -0.4054,  0.4054]], dtype=torch.float64)
tensor([[-0.6757, -0.4054,  0.1351, -1.2162, -0.6757,  0.4054,  1.2162, -0.1351,
         -1.4865,  0.4054,  1.2162,  0.6757]], dtype=torch.float64)
tensor([[ 0.9459,  1.2162, -0.6757, -0.1351,  0.9459, -0.4054, -0.9459,  1.2162,
         -1.7568, -0.4054, -1.4865,  0.9459]], dtype=torch.float64)
tensor([[ 0.1351, -0.4054, -0.6757, -0.6757,  1.2162, -0.1351, -0.4054, -0.4054,
          0.4054, -0.9459, -0.4054,  1.7568]], dtype=torch.float64)
tensor([[ 1.2162, -0.4054,  1.2162,  0.4054, -0.6757, -0.1351,  0.4054, -0.4054,
         -0.4054, -0.4054, -0.1351, -1.2162]], dtype=torch.float64)
tensor([[-0.1351,  0.1351, -0.9459,  0.9459, -1.4865, -0.6757, -0.4054,  2.8378,
         -0.1351, -0.9459, -0.4054,  0.6757]], dtype=torch.float64)
tensor([[ 0.4054,  0.1351, -0.9459,  0

In [ ]:
def rotate(vec, num_rotations): 
    vec2 = np.zeros(len(vec))
    for i in range(len(vec)): 
        vec2[(i + num_rotations )% len(vec)] = vec[i]
    return vec2
data = np.zeros([100,res+1])
for i in range(100):
    SAMPLE_SIZE = 100
    kappa = 1
    vm_dist = vonmises.rvs(kappa, size=SAMPLE_SIZE)
    vm_dist = (vm_dist + np.pi)/(2*np.pi)*(res*10)//10
    
    for k in range(SAMPLE_SIZE):
        data[i][int(vm_dist[k])] += 1
    
    rotate_amt = randint(0,res - 1)
    data[i,0:-1] = rotate(data[i,0:-1], rotate_amt)
    data[i,-1] = -3
np.savetxt("vonmises8ptdata100test.csv", data, delimiter = ",")    

In [79]:
paramlist2 = []
for parameter in net.parameters(): 
    paramlist2 += [parameter]

In [80]:
savednet2 = net

In [81]:
array2 = paramlist2[0].detach().numpy()
array2

array([[ 0.89553136,  0.8492987 ,  0.6373736 ,  0.3802393 , -0.20719768,
        -0.9789834 , -1.1079081 , -1.1494249 , -1.1851345 , -0.84324366,
        -0.09606702,  0.4753184 ],
       [ 0.14079192,  0.6080878 ,  0.9490182 ,  1.050122  ,  0.7022611 ,
         0.3234242 , -0.277926  , -0.55307484, -0.6935756 , -0.6268665 ,
        -0.75529206, -0.3714744 ],
       [ 0.01898059,  0.33570805,  0.4614459 ,  0.6321668 ,  0.37605706,
        -0.23088467,  0.11215439,  0.13479528,  0.01484784,  0.1046114 ,
         0.02233046, -0.11397772],
       [ 0.14834416, -0.10948882,  0.12772757, -0.0285693 , -0.137871  ,
         0.27537763, -0.23182446, -0.13590892, -0.08007377, -0.30612364,
        -0.12377568,  0.02750004],
       [ 0.33975407, -0.2723144 , -0.40492713, -0.6204722 , -0.9514453 ,
        -0.80972844, -0.49622563,  0.29184276,  0.9215568 ,  1.0252419 ,
         0.94372594,  0.7495183 ],
       [-0.9314552 , -0.6436866 , -0.02636125,  0.5785427 ,  0.8592101 ,
         1.0060774 ,  

In [83]:
np.around(np.matmul(array2,fft.ifft(np.identity(12),12)),1)

array([[-0.2+0.j ,  0.5+0.2j,  0. -0.j ,  0. -0.1j,  0. +0.j , -0. +0.j ,
         0. +0.j , -0. -0.j ,  0. -0.j ,  0. +0.1j,  0. +0.j ,  0.5-0.2j],
       [ 0. +0.j ,  0.1+0.4j, -0.1+0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -0.1-0.j ,  0.1-0.4j],
       [ 0.2+0.j ,  0. +0.1j, -0.1+0.1j, -0. -0.j ,  0. +0.1j, -0. -0.j ,
         0. +0.j , -0. +0.j ,  0. -0.1j, -0. +0.j , -0.1-0.1j,  0. -0.1j],
       [-0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j , -0. -0.1j,  0.1+0.j ,
        -0. +0.j ,  0.1-0.j , -0. +0.1j,  0. +0.j ,  0. +0.j ,  0. -0.j ],
       [ 0.1+0.j ,  0.2-0.5j, -0.1+0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0.1-0.j ,  0.2+0.5j],
       [ 0.1+0.j , -0.5+0.2j, -0. +0.j , -0. -0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j , -0. -0.j , -0. +0.j , -0. -0.j , -0.5-0.2j]])

In [77]:
fft.irfft(np.identity(8),8)

array([[ 0.125    ,  0.125    ,  0.125    ,  0.125    ,  0.125    ,
         0.125    ,  0.125    ,  0.125    ],
       [ 0.25     ,  0.1767767,  0.       , -0.1767767, -0.25     ,
        -0.1767767,  0.       ,  0.1767767],
       [ 0.25     ,  0.       , -0.25     ,  0.       ,  0.25     ,
         0.       , -0.25     ,  0.       ],
       [ 0.25     , -0.1767767,  0.       ,  0.1767767, -0.25     ,
         0.1767767,  0.       , -0.1767767],
       [ 0.125    , -0.125    ,  0.125    , -0.125    ,  0.125    ,
        -0.125    ,  0.125    , -0.125    ],
       [ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ],
       [ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ],
       [ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ]])

In [19]:
paramlist2

[Parameter containing:
 tensor([[ 1.8267,  1.6260,  1.0317,  0.1149, -1.1928, -2.3556, -1.1457,  1.0627],
         [-2.6579, -1.6477,  0.0733,  1.0506,  1.7144,  2.1449,  1.0407, -1.2851]],
        requires_grad=True), Parameter containing:
 tensor([0.2029, 0.3341], requires_grad=True), Parameter containing:
 tensor([[ 4.2196,  3.8495],
         [-2.9708, -2.5775]], requires_grad=True), Parameter containing:
 tensor([-6.3560, 11.3028], requires_grad=True)]

In [24]:
net(torch.from_numpy(dataset.__getitem__(1)[0]).float()/20)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [99]:
torch.save(savednet2.state_dict(), "8x6x2bianduniexample.pt")

In [67]:
for i, j in dataset:
    print(sum(i))

1.1102230246251565e-15
-8.881784197001252e-16
1.1102230246251565e-15
-7.771561172376096e-16
-5.551115123125783e-16
4.440892098500626e-16
8.881784197001252e-16
1.7208456881689926e-15
-3.3306690738754696e-15
-1.1102230246251565e-15
-1.9984014443252818e-15
-9.992007221626409e-16
-2.220446049250313e-15
-6.661338147750939e-16
4.440892098500626e-16
3.552713678800501e-15
2.7755575615628914e-15
6.661338147750939e-16
6.661338147750939e-16
-4.440892098500626e-16
1.6653345369377348e-15
-1.1102230246251565e-15
-1.0547118733938987e-15
3.552713678800501e-15
-3.191891195797325e-16
-8.881784197001252e-16
2.220446049250313e-16
-2.220446049250313e-16
-9.992007221626409e-16
2.220446049250313e-16
1.8388068845354155e-15
4.440892098500626e-16
8.881784197001252e-16
3.191891195797325e-16
2.220446049250313e-15
-2.6645352591003757e-15
-1.5543122344752192e-15
1.3322676295501878e-15
1.9984014443252818e-15
-3.1086244689504383e-15
-2.7755575615628914e-15
6.661338147750939e-16
-2.7755575615628914e-15
8.8817841970012

-4.440892098500626e-16
-1.887379141862766e-15
-1.3322676295501878e-15
1.7486012637846216e-15
0.0
1.9984014443252818e-15
-1.1102230246251565e-16
-2.220446049250313e-15
0.0
1.27675647831893e-15
-7.979727989493313e-16
-2.0261570199409107e-15
7.771561172376096e-16
1.1102230246251565e-15
-6.661338147750939e-16
-1.1102230246251565e-16
-2.220446049250313e-16
-4.440892098500626e-16
8.326672684688674e-16
-6.38378239159465e-16
-1.1102230246251565e-16
-1.1102230246251565e-15
-2.275957200481571e-15
2.1094237467877974e-15
-1.4432899320127035e-15
-1.1102230246251565e-16
-5.551115123125783e-17
-3.9968028886505635e-15
-8.881784197001252e-16
1.9012569296705806e-15
1.2212453270876722e-15
-1.0547118733938987e-15
1.3322676295501878e-15
-2.220446049250313e-16
8.881784197001252e-16
-2.7755575615628914e-15
1.1102230246251565e-16
2.6645352591003757e-15
4.440892098500626e-16
-1.1102230246251565e-16
1.6653345369377348e-15
0.0
1.1102230246251565e-15
-2.220446049250313e-15
-2.220446049250313e-16
2.220446049250313

4.440892098500626e-16
2.248201624865942e-15
2.220446049250313e-16
-1.7763568394002505e-15
1.8318679906315083e-15
-3.885780586188048e-16
1.1102230246251565e-15
1.9984014443252818e-15
1.5543122344752192e-15
-3.774758283725532e-15
-4.440892098500626e-16
9.43689570931383e-16
-4.440892098500626e-15
0.0
-1.4710455076283324e-15
-1.1102230246251565e-16
0.0
2.1094237467877974e-15
-2.220446049250313e-16
-4.440892098500626e-16
1.0269562977782698e-15
-8.881784197001252e-16
-2.220446049250313e-16
1.5543122344752192e-15
6.661338147750939e-16
-2.220446049250313e-15
0.0
-2.220446049250313e-15
-1.9984014443252818e-15
-1.887379141862766e-15
2.1094237467877974e-15
-1.2559397966072083e-15
-1.1102230246251565e-15
4.440892098500626e-16
-1.1102230246251565e-15
9.992007221626409e-16
-4.440892098500626e-16
-1.1102230246251565e-15
2.220446049250313e-16
1.5543122344752192e-15
-2.220446049250313e-16
-2.55351295663786e-15
1.1102230246251565e-16
0.0
2.6645352591003757e-15
0.0
6.661338147750939e-16
3.330669073875469

-1.1102230246251565e-16
-2.1649348980190553e-15
3.552713678800501e-15
1.4432899320127035e-15
-1.1102230246251565e-15
8.881784197001252e-16
8.881784197001252e-16
-2.4424906541753444e-15
-7.771561172376096e-16
-1.6653345369377348e-15
-9.992007221626409e-16
-1.9984014443252818e-15
1.27675647831893e-15
-1.3600232051658168e-15
2.6645352591003757e-15
1.7763568394002505e-15
-6.661338147750939e-16
8.881784197001252e-16
1.1102230246251565e-16
-1.5543122344752192e-15
-2.220446049250313e-16
1.304512053934559e-15
0.0
1.6583956430338276e-15
9.992007221626409e-16
1.3322676295501878e-15
-2.609024107869118e-15
0.0
6.661338147750939e-16
-1.1102230246251565e-16
8.881784197001252e-16
-1.033895191682177e-15
1.9984014443252818e-15
-2.4424906541753444e-15
2.220446049250313e-16
0.0
1.9984014443252818e-15
-2.1094237467877974e-15
0.0
2.0816681711721685e-15
1.304512053934559e-15
2.4980018054066022e-15
2.7755575615628914e-15
-2.4424906541753444e-15
1.2212453270876722e-15
2.6645352591003757e-15
-7.771561172376096

-6.661338147750939e-16
2.7755575615628914e-17
-4.440892098500626e-16
3.1086244689504383e-15
1.1102230246251565e-15
1.8318679906315083e-15
-1.1102230246251565e-15
-2.6818824938601438e-15
-3.885780586188048e-16
-1.3877787807814457e-15
1.7763568394002505e-15
-2.220446049250313e-16
2.3592239273284576e-16
2.220446049250313e-15
0.0
-2.220446049250313e-16
8.881784197001252e-16
-1.1102230246251565e-16
-1.3877787807814457e-15
1.1102230246251565e-15
-1.5543122344752192e-15
-6.661338147750939e-16
1.1102230246251565e-15
-9.992007221626409e-16
6.661338147750939e-16
1.9984014443252818e-15
2.4424906541753444e-15
-2.220446049250313e-15
2.220446049250313e-16
-1.7763568394002505e-15
1.1102230246251565e-15
-1.5543122344752192e-15
-1.5543122344752192e-15
0.0
1.4432899320127035e-15
-1.5543122344752192e-15
-1.4988010832439613e-15
9.159339953157541e-16
-8.881784197001252e-16
1.887379141862766e-15
8.881784197001252e-16
-2.220446049250313e-16
1.5543122344752192e-15
-7.771561172376096e-16
7.7021722333370235e-16

...

-2.4424906541753444e-15
8.881784197001252e-16
-3.1086244689504383e-15
4.440892098500626e-16
-3.3306690738754696e-16
0.0
1.1171619185290638e-15
4.440892098500626e-16
-1.1102230246251565e-16
4.440892098500626e-16
-8.881784197001252e-16
-1.7486012637846216e-15
2.7755575615628914e-15
1.3322676295501878e-15
8.881784197001252e-16
4.440892098500626e-16
-2.942091015256665e-15
1.1102230246251565e-15
1.6653345369377348e-15
-1.3322676295501878e-15
-1.5543122344752192e-15
-6.661338147750939e-16
-1.1449174941446927e-15
0.0
-1.5543122344752192e-15
0.0
-8.881784197001252e-16
2.55351295663786e-15
-2.4424906541753444e-15
4.440892098500626e-16
1.4432899320127035e-15
4.440892098500626e-16
-2.6645352591003757e-15
-2.4147350785597155e-15
1.4988010832439613e-15
0.0
1.3322676295501878e-15
2.6367796834847468e-15
6.661338147750939e-16
2.220446049250313e-16
5.551115123125783e-17
1.7208456881689926e-15
1.2212453270876722e-15
-2.4424906541753444e-15
1.7763568394002505e-15
-4.107825191113079e-15
2.498001805406602e

-1.9984014443252818e-15
2.220446049250313e-16
-4.440892098500626e-16
5.551115123125783e-17
-9.992007221626409e-16
2.7200464103316335e-15
-1.3322676295501878e-15
1.9984014443252818e-15
-1.1102230246251565e-16
3.3306690738754696e-16
-8.881784197001252e-16
-7.771561172376096e-16
1.4988010832439613e-15
1.0547118733938987e-15
6.661338147750939e-16
-5.551115123125783e-16
1.7763568394002505e-15
-1.7763568394002505e-15
2.220446049250313e-16
2.1094237467877974e-15
0.0
-1.2351231148954867e-15
2.6645352591003757e-15
1.5543122344752192e-15
4.440892098500626e-16
4.996003610813204e-16
-1.1102230246251565e-16
-1.9984014443252818e-15
-2.1649348980190553e-15
-1.9984014443252818e-15
-2.3314683517128287e-15
6.661338147750939e-16
-8.881784197001252e-16
5.551115123125783e-16
1.1657341758564144e-15
-4.440892098500626e-16
2.7755575615628914e-17
-1.5543122344752192e-15
-4.440892098500626e-16
-8.881784197001252e-16
1.304512053934559e-15
3.0808688933348094e-15
1.3322676295501878e-15
2.220446049250313e-16
-1.554

1.1102230246251565e-16
4.440892098500626e-16
1.609823385706477e-15
1.5543122344752192e-15
2.220446049250313e-16
-4.440892098500626e-16
-1.9984014443252818e-15
-2.6645352591003757e-15
3.0531133177191805e-16
0.0
0.0
2.0539125955565396e-15
6.661338147750939e-16
1.887379141862766e-15
-3.3306690738754696e-15
4.440892098500626e-16
1.9984014443252818e-15
1.3322676295501878e-15
-1.6653345369377348e-15
2.220446049250313e-16
4.440892098500626e-16
4.440892098500626e-16
-2.275957200481571e-15
-1.7763568394002505e-15
3.552713678800501e-15
2.5396351688300456e-15
-1.9984014443252818e-15
-2.6367796834847468e-15
-6.661338147750939e-16
2.220446049250313e-16
-3.1086244689504383e-15
-2.55351295663786e-15
5.551115123125783e-16
-9.992007221626409e-16
-2.4424906541753444e-15
-4.440892098500626e-16
0.0
-1.7763568394002505e-15
-3.4416913763379853e-15
-1.5543122344752192e-15
-6.661338147750939e-16
2.4424906541753444e-15
1.6861512186494565e-15
-2.220446049250313e-16
1.6514567491299204e-15
1.1102230246251565e-15


4.440892098500626e-16
2.6645352591003757e-15
3.191891195797325e-15
-1.9984014443252818e-15
-4.440892098500626e-16
-2.220446049250313e-15
-2.220446049250313e-15
-1.5543122344752192e-15
1.9984014443252818e-15
-1.7763568394002505e-15
8.881784197001252e-16
-2.220446049250313e-15
2.886579864025407e-15
-1.887379141862766e-15
-2.3314683517128287e-15
2.0539125955565396e-15
-1.2212453270876722e-15
2.220446049250313e-15
8.881784197001252e-16
-1.1102230246251565e-16
1.5543122344752192e-15
4.440892098500626e-16
1.3322676295501878e-15
-8.881784197001252e-16
6.661338147750939e-16
2.55351295663786e-15
-4.440892098500626e-16
-2.4424906541753444e-15
1.3322676295501878e-15
6.661338147750939e-16
-1.3322676295501878e-15
-2.7755575615628914e-16
0.0
2.498001805406602e-16
-2.220446049250313e-16
-2.220446049250313e-16
-1.762479051592436e-15
-1.4432899320127035e-15
1.3877787807814457e-15
1.1102230246251565e-16
0.0
-1.1102230246251565e-15
8.881784197001252e-16
6.38378239159465e-16
-7.216449660063518e-16
1.42941

KeyboardInterrupt: 